# Import libraries

In [1]:
import requests
import pandas as pd

# Import Agents

In [2]:
from DummyAgent import DummyAgent
from LinRegAgent import LinRegAgent
from CatBoostAgent import CatBoostAgent
from LstmAgent import LstmAgent
from QAgent import QLearningAgent

# Data Collection

In [3]:
def fetch_historical_data(symbol, interval, limit=1000):
    url = f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&limit={limit}'
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
    return df

In [4]:
# Fetch data for different intervals
df_1m = fetch_historical_data('BTCUSDT', '1m')
df_1h = fetch_historical_data('BTCUSDT', '1h')
df_4h = fetch_historical_data('BTCUSDT', '4h')
df_1d = fetch_historical_data('BTCUSDT', '1d')

# Data Preparation

In [5]:
# Prepare the training data
def prepare_data(data):
    data['returns'] = data['close'].pct_change()
    data.dropna(inplace=True)
    return data

In [6]:
df_1m = prepare_data(df_1m)
df_1h = prepare_data(df_1h)
df_4h = prepare_data(df_4h)
df_1d = prepare_data(df_1d)

# Backtesting

In [7]:
# Backtesting function
def backtest(agent, data):
    agent.train_model(data)
    for timestamp, row in data.iterrows():
        agent.trade(data.loc[:timestamp])

In [8]:
# Initialize agents
dummy_agent_1m = DummyAgent('Dummy Agent 1m')
dummy_agent_1h = DummyAgent('Dummy Agent 1h')
dummy_agent_4h = DummyAgent('Dummy Agent 4h')
dummy_agent_1d = DummyAgent('Dummy Agent 1d')

lr_agent_1m_price = LinRegAgent('Linear Regression Agent 1m (Price)', 'close')
lr_agent_1h_price = LinRegAgent('Linear Regression Agent 1h (Price)', 'close')
lr_agent_4h_price = LinRegAgent('Linear Regression Agent 4h (Price)', 'close')
lr_agent_1d_price = LinRegAgent('Linear Regression Agent 1d (Price)', 'close')

lr_agent_1m_returns = LinRegAgent('Linear Regression Agent 1m (Returns)', 'returns', buy_threshold=0.001, sell_threshold=0.0015)
lr_agent_1h_returns = LinRegAgent('Linear Regression Agent 1h (Returns)', 'returns', buy_threshold=0.01, sell_threshold=0.02)
lr_agent_4h_returns = LinRegAgent('Linear Regression Agent 4h (Returns)', 'returns', buy_threshold=0.012, sell_threshold=0.03)
lr_agent_1d_returns = LinRegAgent('Linear Regression Agent 1d (Returns)', 'returns', buy_threshold=0.05, sell_threshold=0.06)

cb_agent_1m_price = CatBoostAgent('CatBoost Agent 1m (Price)', 'close')
cb_agent_1h_price = CatBoostAgent('CatBoost Agent 1h (Price)', 'close')
cb_agent_4h_price = CatBoostAgent('CatBoost Agent 4h (Price)', 'close')
cb_agent_1d_price = CatBoostAgent('CatBoost Agent 1d (Price)', 'close')

cb_agent_1m_returns = CatBoostAgent('CatBoost Agent 1m (Returns)', 'returns', buy_threshold=0.001, sell_threshold=0.0015)
cb_agent_1h_returns = CatBoostAgent('CatBoost Agent 1h (Returns)', 'returns', buy_threshold=0.01, sell_threshold=0.02)
cb_agent_4h_returns = CatBoostAgent('CatBoost Agent 4h (Returns)', 'returns', buy_threshold=0.012, sell_threshold=0.03)
cb_agent_1d_returns = CatBoostAgent('CatBoost Agent 1d (Returns)', 'returns', buy_threshold=0.05, sell_threshold=0.06)

lstm_agent_1m_price = LstmAgent('LSTM Agent 1m (Price)')
lstm_agent_1h_price = LstmAgent('LSTM Agent 1h (Price)')
lstm_agent_4h_price = LstmAgent('LSTM Agent 4h (Price)')
lstm_agent_1d_price = LstmAgent('LSTM Agent 1d (Price)')

lstm_agent_1m_returns = LstmAgent('LSTM Agent 1m (Returns)')
lstm_agent_1h_returns = LstmAgent('LSTM Agent 1h (Returns)')
lstm_agent_4h_returns = LstmAgent('LSTM Agent 4h (Returns)')
lstm_agent_1d_returns = LstmAgent('LSTM Agent 1d (Returns)')

ql_agent_1m = QLearningAgent('Q-learning Agent 1m')
ql_agent_1h = QLearningAgent('Q-learning Agent 1h')
ql_agent_4h = QLearningAgent('Q-learning Agent 4h')
ql_agent_1d = QLearningAgent('Q-learning Agent 1d')

In [9]:
# Backtest each agent
backtest(dummy_agent_1m, df_1m)
backtest(dummy_agent_1h, df_1h)
backtest(dummy_agent_4h, df_4h)
backtest(dummy_agent_1d, df_1d)

backtest(lr_agent_1m_price, df_1m)
backtest(lr_agent_1h_price, df_1h)
backtest(lr_agent_4h_price, df_4h)
backtest(lr_agent_1d_price, df_1d)

backtest(lr_agent_1m_returns, df_1m)
backtest(lr_agent_1h_returns, df_1h)
backtest(lr_agent_4h_returns, df_4h)
backtest(lr_agent_1d_returns, df_1d)

backtest(cb_agent_1m_price, df_1m)
backtest(cb_agent_1h_price, df_1h)
backtest(cb_agent_4h_price, df_4h)
backtest(cb_agent_1d_price, df_1d)

backtest(cb_agent_1m_returns, df_1m)
backtest(cb_agent_1h_returns, df_1h)
backtest(cb_agent_4h_returns, df_4h)
backtest(cb_agent_1d_returns, df_1d)

backtest(lstm_agent_1m_price, df_1m)
backtest(lstm_agent_1h_price, df_1h)
backtest(lstm_agent_4h_price, df_4h)
backtest(lstm_agent_1d_price, df_1d)

backtest(lstm_agent_1m_returns, df_1m)
backtest(lstm_agent_1h_returns, df_1h)
backtest(lstm_agent_4h_returns, df_4h)
backtest(lstm_agent_1d_returns, df_1d)

backtest(ql_agent_1m, df_1m)
backtest(ql_agent_1h, df_1h)
backtest(ql_agent_4h, df_4h)
backtest(ql_agent_1d, df_1d)

2024-07-24 06:31:00: Dummy Agent 1m - Buy at 65913.99
2024-07-24 06:32:00: Dummy Agent 1m - Hold
2024-07-24 06:33:00: Dummy Agent 1m - Sell at 65897.14
2024-07-24 06:34:00: Dummy Agent 1m - Buy at 65901.92
2024-07-24 06:35:00: Dummy Agent 1m - Hold
2024-07-24 06:36:00: Dummy Agent 1m - Sell at 65915.1
2024-07-24 06:37:00: Dummy Agent 1m - Hold
2024-07-24 06:38:00: Dummy Agent 1m - Hold
2024-07-24 06:39:00: Dummy Agent 1m - Buy at 65874.0
2024-07-24 06:40:00: Dummy Agent 1m - Hold
2024-07-24 06:41:00: Dummy Agent 1m - Hold
2024-07-24 06:42:00: Dummy Agent 1m - Hold
2024-07-24 06:43:00: Dummy Agent 1m - Hold
2024-07-24 06:44:00: Dummy Agent 1m - Hold
2024-07-24 06:45:00: Dummy Agent 1m - Hold
2024-07-24 06:46:00: Dummy Agent 1m - Sell at 65832.49
2024-07-24 06:47:00: Dummy Agent 1m - Buy at 65900.0
2024-07-24 06:48:00: Dummy Agent 1m - Hold
2024-07-24 06:49:00: Dummy Agent 1m - Sell at 65901.7
2024-07-24 06:50:00: Dummy Agent 1m - Buy at 65882.01
2024-07-24 06:51:00: Dummy Agent 1m - Sel

/Users/puchit/Desktop/Work/SIIT/CSS483/Algorithmic-Trading/src/LstmAgent.py:37: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = data[i+window_size]


32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.1998 - root_mean_squared_error: 0.4340
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0107 - root_mean_squared_error: 0.1034
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0087 - root_mean_squared_error: 0.0929
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0094 - root_mean_squared_error: 0.0971
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0094 - root_mean_squared_error: 0.0970
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0087 - root_mean_squared_error: 0.0930
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0089 - root_mean_squared_error: 0.0943
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0089 - root_mean_squared_error: 0.0939
Epoch 9/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0090 - root_mean_squared_error: 0.0948
Epoch 10/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0080 - root_mean_squared_error: 0.0892
Epoch 11/5

/Users/puchit/Desktop/Work/SIIT/CSS483/Algorithmic-Trading/src/LstmAgent.py:37: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = data[i+window_size]


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0234 - root_mean_squared_error: 0.1518
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0159 - root_mean_squared_error: 0.1259
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.0129 - root_mean_squared_error: 0.1134
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0158 - root_mean_squared_error: 0.1254
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0150 - root_mean_squared_error: 0.1219
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0134 - root_mean_squared_error: 0.1154
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0120 - root_mean_squared_error: 0.1093
Epoch 8/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0144 - root_mean_squared_error: 0.1198
Epoch 9/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0119 - root_mean_squared_error: 0.1091
Epoch 10/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0134 - root_mean_squared_error: 0.1159
Epoch 11/5

...

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0080 - root_mean_squared_error: 0.0892
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0094 - root_mean_squared_error: 0.0971
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0094 - root_mean_squared_error: 0.0969
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0103 - root_mean_squared_error: 0.1005
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0095 - root_mean_squared_error: 0.0975
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0079 - root_mean_squared_error: 0.0882
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0079 - root_mean_squared_error: 0.0888
Epoch 8/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0105 - root_mean_squared_error: 0.1023
Epoch 9/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0094 - root_mean_squared_error: 0.0969
Epoch 10/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0099 - root_mean_squared_error: 0.0992
Epoch 11/5

/Users/puchit/Desktop/Work/SIIT/CSS483/Algorithmic-Trading/src/LstmAgent.py:37: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = data[i+window_size]


32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.1002 - root_mean_squared_error: 0.3067
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0144 - root_mean_squared_error: 0.1198
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0110 - root_mean_squared_error: 0.1045
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0109 - root_mean_squared_error: 0.1045
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0120 - root_mean_squared_error: 0.1093
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0119 - root_mean_squared_error: 0.1088
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0102 - root_mean_squared_error: 0.1005
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0109 - root_mean_squared_error: 0.1042
Epoch 9/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0110 - root_mean_squared_error: 0.1049
Epoch 10/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0100 - root_mean_squared_error: 0.0999 
Epoch 11/50
32/

/Users/puchit/Desktop/Work/SIIT/CSS483/Algorithmic-Trading/src/LstmAgent.py:37: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = data[i+window_size]


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0115 - root_mean_squared_error: 0.1067
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0111 - root_mean_squared_error: 0.1052
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0122 - root_mean_squared_error: 0.1101
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0115 - root_mean_squared_error: 0.1072
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0135 - root_mean_squared_error: 0.1157
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0119 - root_mean_squared_error: 0.1086
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0106 - root_mean_squared_error: 0.1025
Epoch 8/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0123 - root_mean_squared_error: 0.1104
Epoch 9/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0124 - root_mean_squared_error: 0.1114
Epoch 10/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0122 - root_mean_squared_error: 0.1104
Epoch 11/50
16

In [24]:
def evaluate_agent(agent):
    if '1m' in agent.name:
        data = df_1m['close']
    elif '1h' in agent.name:
        data = df_1h['close']
    elif '4h' in agent.name:
        data = df_4h['close']
    elif '1d' in agent.name:
        data = df_1d['close']
    port_value = agent.get_portfolio_value(data.iloc[-1])
    total_return = (port_value - 100000) / 100000 * 100
    sharpe_ratio = total_return / data.std()
    print(f'----- {agent.name} -----')
    print(f"Portfolio Value for {agent.name}: {port_value:.6f}")
    print(f"Total Return for {agent.name}: {total_return:.6f}%")
    print(f"Sharpe Ratio for {agent.name}: {sharpe_ratio:.6f}")
    print(f"Maximum Drawdown for {agent.name}: {agent.get_max_drawdown():.6f}%")
    print()

In [25]:
evaluate_agent(dummy_agent_1m)
evaluate_agent(dummy_agent_1h)
evaluate_agent(dummy_agent_4h)
evaluate_agent(dummy_agent_1d)

----- Dummy Agent 1m -----
Portfolio Value for Dummy Agent 1m: 100726.796710
Total Return for Dummy Agent 1m: 0.726797%
Sharpe Ratio for Dummy Agent 1m: 0.001901
Maximum Drawdown for Dummy Agent 1m: 1.732269%

----- Dummy Agent 1h -----
Portfolio Value for Dummy Agent 1h: 101515.200841
Total Return for Dummy Agent 1h: 1.515201%
Sharpe Ratio for Dummy Agent 1h: 0.000429
Maximum Drawdown for Dummy Agent 1h: 9.662914%

----- Dummy Agent 4h -----
Portfolio Value for Dummy Agent 4h: 118499.935773
Total Return for Dummy Agent 4h: 18.499936%
Sharpe Ratio for Dummy Agent 4h: 0.003193
Maximum Drawdown for Dummy Agent 4h: 20.829742%

----- Dummy Agent 1d -----
Portfolio Value for Dummy Agent 1d: 40154.283492
Total Return for Dummy Agent 1d: -59.845717%
Sharpe Ratio for Dummy Agent 1d: -0.003757
Maximum Drawdown for Dummy Agent 1d: 70.004875%



In [26]:
evaluate_agent(lr_agent_1m_price)
evaluate_agent(lr_agent_1h_price)
evaluate_agent(lr_agent_4h_price)
evaluate_agent(lr_agent_1d_price)
print()
evaluate_agent(lr_agent_1m_returns)
evaluate_agent(lr_agent_1h_returns)
evaluate_agent(lr_agent_4h_returns)
evaluate_agent(lr_agent_1d_returns)

----- Linear Regression Agent 1m (Price) -----
Portfolio Value for Linear Regression Agent 1m (Price): 99132.516532
Total Return for Linear Regression Agent 1m (Price): -0.867483%
Sharpe Ratio for Linear Regression Agent 1m (Price): -0.002269
Maximum Drawdown for Linear Regression Agent 1m (Price): 2.032292%

----- Linear Regression Agent 1h (Price) -----
Portfolio Value for Linear Regression Agent 1h (Price): 110975.230157
Total Return for Linear Regression Agent 1h (Price): 10.975230%
Sharpe Ratio for Linear Regression Agent 1h (Price): 0.003111
Maximum Drawdown for Linear Regression Agent 1h (Price): 8.834007%

----- Linear Regression Agent 4h (Price) -----
Portfolio Value for Linear Regression Agent 4h (Price): 103607.118974
Total Return for Linear Regression Agent 4h (Price): 3.607119%
Sharpe Ratio for Linear Regression Agent 4h (Price): 0.000623
Maximum Drawdown for Linear Regression Agent 4h (Price): 19.178515%

----- Linear Regression Agent 1d (Price) -----
Portfolio Value for 

In [27]:
evaluate_agent(cb_agent_1m_price)
evaluate_agent(cb_agent_1h_price)
evaluate_agent(cb_agent_4h_price)
evaluate_agent(cb_agent_1d_price)
print()
evaluate_agent(cb_agent_1m_returns)
evaluate_agent(cb_agent_1h_returns)
evaluate_agent(cb_agent_4h_returns)
evaluate_agent(cb_agent_1d_returns)

----- CatBoost Agent 1m (Price) -----
Portfolio Value for CatBoost Agent 1m (Price): 99117.496984
Total Return for CatBoost Agent 1m (Price): -0.882503%
Sharpe Ratio for CatBoost Agent 1m (Price): -0.002308
Maximum Drawdown for CatBoost Agent 1m (Price): 1.372296%

----- CatBoost Agent 1h (Price) -----
Portfolio Value for CatBoost Agent 1h (Price): 104709.981104
Total Return for CatBoost Agent 1h (Price): 4.709981%
Sharpe Ratio for CatBoost Agent 1h (Price): 0.001335
Maximum Drawdown for CatBoost Agent 1h (Price): 10.761943%

----- CatBoost Agent 4h (Price) -----
Portfolio Value for CatBoost Agent 4h (Price): 93847.535835
Total Return for CatBoost Agent 4h (Price): -6.152464%
Sharpe Ratio for CatBoost Agent 4h (Price): -0.001062
Maximum Drawdown for CatBoost Agent 4h (Price): 19.837128%

----- CatBoost Agent 1d (Price) -----
Portfolio Value for CatBoost Agent 1d (Price): 114092.810106
Total Return for CatBoost Agent 1d (Price): 14.092810%
Sharpe Ratio for CatBoost Agent 1d (Price): 0.0

In [28]:
evaluate_agent(lstm_agent_1m_price)
evaluate_agent(lstm_agent_1h_price)
evaluate_agent(lstm_agent_4h_price)
evaluate_agent(lstm_agent_1d_price)
print()
evaluate_agent(lstm_agent_1m_returns)
evaluate_agent(lstm_agent_1h_returns)
evaluate_agent(lstm_agent_4h_returns)
evaluate_agent(lstm_agent_1d_returns)

----- LSTM Agent 1m (Price) -----
Portfolio Value for LSTM Agent 1m (Price): 98450.550577
Total Return for LSTM Agent 1m (Price): -1.549449%
Sharpe Ratio for LSTM Agent 1m (Price): -0.004053
Maximum Drawdown for LSTM Agent 1m (Price): 2.418709%

----- LSTM Agent 1h (Price) -----
Portfolio Value for LSTM Agent 1h (Price): 97194.672093
Total Return for LSTM Agent 1h (Price): -2.805328%
Sharpe Ratio for LSTM Agent 1h (Price): -0.000795
Maximum Drawdown for LSTM Agent 1h (Price): 8.032539%

----- LSTM Agent 4h (Price) -----
Portfolio Value for LSTM Agent 4h (Price): 100993.241488
Total Return for LSTM Agent 4h (Price): 0.993241%
Sharpe Ratio for LSTM Agent 4h (Price): 0.000171
Maximum Drawdown for LSTM Agent 4h (Price): 10.113818%

----- LSTM Agent 1d (Price) -----
Portfolio Value for LSTM Agent 1d (Price): 163232.891488
Total Return for LSTM Agent 1d (Price): 63.232891%
Sharpe Ratio for LSTM Agent 1d (Price): 0.003969
Maximum Drawdown for LSTM Agent 1d (Price): 9.985683%


----- LSTM Agen

In [29]:
evaluate_agent(ql_agent_1m)
evaluate_agent(ql_agent_1h)
evaluate_agent(ql_agent_4h)
evaluate_agent(ql_agent_1d)


----- Q-learning Agent 1m -----
Portfolio Value for Q-learning Agent 1m: 100323.652378
Total Return for Q-learning Agent 1m: 0.323652%
Sharpe Ratio for Q-learning Agent 1m: 0.000847
Maximum Drawdown for Q-learning Agent 1m: 1.484685%

----- Q-learning Agent 1h -----
Portfolio Value for Q-learning Agent 1h: 113203.366365
Total Return for Q-learning Agent 1h: 13.203366%
Sharpe Ratio for Q-learning Agent 1h: 0.003742
Maximum Drawdown for Q-learning Agent 1h: 3.743923%

----- Q-learning Agent 4h -----
Portfolio Value for Q-learning Agent 4h: 103085.859298
Total Return for Q-learning Agent 4h: 3.085859%
Sharpe Ratio for Q-learning Agent 4h: 0.000533
Maximum Drawdown for Q-learning Agent 4h: 18.124506%

----- Q-learning Agent 1d -----
Portfolio Value for Q-learning Agent 1d: 167642.726049
Total Return for Q-learning Agent 1d: 67.642726%
Sharpe Ratio for Q-learning Agent 1d: 0.004246
Maximum Drawdown for Q-learning Agent 1d: 26.000724%

